In [1]:
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation


!pip install geopy
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize


! pip install folium==0.5.0
import folium # plotting library

print('Folium installed')
print('Libraries imported.')

Folium installed
Libraries imported.


In [40]:
CLIENT_ID = '354Y3HRRFNJHWDIFEDF4SE4KB0GNAFFTUAS4IERNRAZLSNIU' # your Foursquare ID
CLIENT_SECRET = '5URO22QW0Y2POVS1FLKPNURZVW4XDHS4KX0P50NQKVTQWYNO' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 354Y3HRRFNJHWDIFEDF4SE4KB0GNAFFTUAS4IERNRAZLSNIU
CLIENT_SECRET:5URO22QW0Y2POVS1FLKPNURZVW4XDHS4KX0P50NQKVTQWYNO


In [107]:
search_query = 'Downtown Toronto'
radius = 500
print(search_query + ' .... OK!')

Downtown Toronto .... OK!


In [108]:
latitude = 43.654260
longitude = -79.360636

In [109]:
url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?client_id=354Y3HRRFNJHWDIFEDF4SE4KB0GNAFFTUAS4IERNRAZLSNIU&client_secret=5URO22QW0Y2POVS1FLKPNURZVW4XDHS4KX0P50NQKVTQWYNO&ll=43.65426,-79.360636&v=20180604&radius=500&limit=30'

In [110]:
import requests

In [111]:
results = requests.get(url).json()
'There are {} neighbourhood around East Toronto.'.format(len(results['response']['groups'][0]['items']))

'There are 30 neighbourhood around East Toronto.'

In [112]:
items = results['response']['groups'][0]['items']
items[0]

{'reasons': {'count': 0,
  'items': [{'summary': 'This spot is popular',
    'type': 'general',
    'reasonName': 'globalInteractionReason'}]},
 'venue': {'id': '54ea41ad498e9a11e9e13308',
  'name': 'Roselle Desserts',
  'location': {'address': '362 King St E',
   'crossStreet': 'Trinity St',
   'lat': 43.653446723052674,
   'lng': -79.3620167174383,
   'labeledLatLngs': [{'label': 'display',
     'lat': 43.653446723052674,
     'lng': -79.3620167174383}],
   'distance': 143,
   'postalCode': 'M5A 1K9',
   'cc': 'CA',
   'city': 'Toronto',
   'state': 'ON',
   'country': 'Canada',
   'formattedAddress': ['362 King St E (Trinity St)',
    'Toronto ON M5A 1K9',
    'Canada']},
  'categories': [{'id': '4bf58dd8d48988d16a941735',
    'name': 'Bakery',
    'pluralName': 'Bakeries',
    'shortName': 'Bakery',
    'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/bakery_',
     'suffix': '.png'},
    'primary': True}],
  'photos': {'count': 0, 'groups': []}},
 'referralId': 'e-0

In [113]:
dataframe = json_normalize(items) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories'] + [col for col in dataframe.columns if col.startswith('venue.location.')] + ['venue.id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['venue.categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean columns
dataframe_filtered.columns = [col.split('.')[-1] for col in dataframe_filtered.columns]

dataframe_filtered.head(10)

C:\Users\Bin-Bibo-Uno\anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  """Entry point for launching an IPython kernel.


,name,categories,address,crossStreet,lat,lng,labeledLatLngs,distance,postalCode,cc,city,state,country,formattedAddress,neighborhood,id
0,Roselle Desserts,Bakery,362 King St E,Trinity St,43.653447,-79.362017,"[{'label': 'display', 'lat': 43.65344672305267...",143,M5A 1K9,CA,Toronto,ON,Canada,"[362 King St E (Trinity St), Toronto ON M5A 1K...",NaN,54ea41ad498e9a11e9e13308
1,Tandem Coffee,Coffee Shop,368 King St E,at Trinity St,43.653559,-79.361809,"[{'label': 'display', 'lat': 43.65355870959944...",122,NaN,CA,Toronto,ON,Canada,"[368 King St E (at Trinity St), Toronto ON, Ca...",NaN,53b8466a498e83df908c3f21
2,Cooper Koo Family YMCA,Distribution Center,461 Cherry St,NaN,43.653249,-79.358008,"[{'label': 'display', 'lat': 43.65324910177244...",239,M5A 0H7,CA,Toronto,ON,Canada,"[461 Cherry St, Toronto ON M5A 0H7, Canada]",NaN,574c229e498ebb5c6b257902
3,Body Blitz Spa East,Spa,497 King Street East,btwn Sackville St and Sumach St,43.654735,-79.359874,"[{'label': 'display', 'lat': 43.65473505045365...",80,M5A 1L9,CA,Toronto,ON,Canada,[497 King Street East (btwn Sackville St and S...,NaN,50760559e4b0e8c7babe2497
4,Impact Kitchen,Restaurant,573 King St E,at St Lawrence St,43.656369,-79.356980,"[{'label': 'display', 'lat': 43.65636850543279...",376,M5A 4L3,CA,Toronto,ON,Canada,"[573 King St E (at St Lawrence St), Toronto ON...",NaN,5612b1cc498e3dd742af0dc8
5,Corktown Common,Park,NaN,NaN,43.655618,-79.356211,"[{'label': 'display', 'lat': 43.65561779974973...",387,NaN,CA,NaN,NaN,Canada,[Canada],NaN,51ccc048498ec7792efc955e
6,Dominion Pub and Kitchen,Pub,500 Queen Street East,NaN,43.656919,-79.358967,"[{'label': 'display', 'lat': 43.65691857501867...",325,M5A 1T9,CA,Toronto,ON,Canada,"[500 Queen Street East, Toronto ON M5A 1T9, Ca...",NaN,566e1294498e3f6629006bc3
7,The Distillery Historic District,Historic Site,"btwn Front, Cherry, Gardiner & Parliament",NaN,43.650244,-79.359323,"[{'label': 'display', 'lat': 43.65024435658077...",459,M5A 3C4,CA,Toronto,ON,Canada,"[btwn Front, Cherry, Gardiner & Parliament, To...",NaN,4ad4c05ef964a520bff620e3
8,Morning Glory Cafe,Breakfast Spot,457 King St. E,Gilead Place,43.653947,-79.361149,"[{'label': 'display', 'lat': 43.65394694263529...",54,M5A 1L6,CA,Toronto,ON,Canada,"[457 King St. E (Gilead Place), Toronto ON M5A...",NaN,4ae5b91ff964a520a6a121e3
9,The Extension Room,Gym / Fitness Center,30 Eastern Ave,Sackville St.,43.653313,-79.359725,"[{'label': 'display', 'lat': 43.65331304337331...",128,NaN,CA,Toronto,ON,Canada,"[30 Eastern Ave (Sackville St.), Toronto ON, C...",NaN,4e8b7fa1cc2112f67517660a


In [114]:
venues_map = folium.Map(location=[latitude, longitude], zoom_start=15) # generate map centred around Ecco


# add Ecco as a red circle mark
folium.CircleMarker(
    [latitude, longitude],
    radius=10,
    popup='Ecco',
    fill=True,
    color='red',
    fill_color='red',
    fill_opacity=0.6
    ).add_to(venues_map)


# add popular spots to the map as blue circle markers
for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.categories):
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        fill=True,
        color='blue',
        fill_color='blue',
        fill_opacity=0.6
        ).add_to(venues_map)

# display map
venues_map